<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive-into-Deep-Learning-Notes-/blob/main/6_builderguide_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PyTorch Notes

* `torch.rand()`: Returns a tensor filled with random numbers from a uniform distribution on the interval [0, 1).

In [3]:
import torch

X = torch.rand(size=(2, 2))
X

tensor([[0.3017, 0.5035],
        [0.9197, 0.4851]])

* `add_module()` method in `Module` class: adds a child module to the current module. This method is useful when adding modules using the `for` loop.
* `children()` method in `Module` class: Returns an iterator over *immediate children* modules. This method is a generator that returns layers of the model from which you can extract parameter tensors using `layername.wieght` and `layername.bias`.

In [28]:
# add_module
from torch import nn

X = torch.rand(2, 20)

modules1 = {'linear1': nn.LazyLinear(256),
           'actv1': nn.ReLU(),
           'linear2': nn.LazyLinear(10)}

class Net(nn.Module):
  def __init__(self, **kwargs):
    super().__init__()

    for key, value in kwargs.items():
      self.add_module(key, value)

  def forward(self, X):
    for module in self.children():         # returns the modules of a neural network ,modules() will not work here because it considers all the modules (the net plus three others)
      X = module(X)
    return X

net = Net(**modules1)

for module in net.modules():
  print(module)
  break          


Net(
  (linear1): LazyLinear(in_features=0, out_features=256, bias=True)
  (actv1): ReLU()
  (linear2): LazyLinear(in_features=0, out_features=10, bias=True)
)


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


* `modules()` method in `Module` class: Returns an iterator over *all modules* in the network. If we want to recursively iterate over modules, then we should use `modules()` method instead of `children()` method.

## General Notes

### \*args and \**kwargs

* \*args: **Non-Keyword Arguments**: This is used in function definition is python and is used to pass a number of arguments to a function. 
* \*\*kwargs: **Keyword Arguments**: Is used in function definition in Python and is used to pass a *keyworded*, variable-length argument list.

* A keyword argument is where you provide a name to the variable as you pass it into the function. kwargs can be think of as *dictionary*.

In [5]:
# *args
def myfunc(*args):
  for arg in args:
    print(arg)

args = ('Davood', 'Ahmad', 'Akbar', 'Mohsen')
myfunc(*args)

Davood
Ahmad
Akbar
Mohsen


In [6]:
# **kwargs

def myfunc1(**kwargs):
  for key, value in kwargs.items():         # Items() method returns the key and value in a dictionary.
    print(f'{key} == {value}')

myfunc1(name='Davood', age=29, education='M.Sc.')

name == Davood
age == 29
education == M.Sc.
